In [1]:
!pip install camelot-py[all]
!pip install tabula-py
!pip install PyPDF2
!apt-get install ghostscript

     |████████████████████████████████| 40 kB 13 kB/s 
     |████████████████████████████████| 5.6 MB 24.2 MB/s 
     |████████████████████████████████| 77 kB 4.5 MB/s 
     |████████████████████████████████| 11.7 MB 141 kB/s 
     |████████████████████████████████| 3.5 MB 48.6 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=7ae1a37a31e2e4d982b60ea8dda7e938450136db80674e88e89b61d60b94e962
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
     |████████████████████████████████| 12.0 MB 21.8 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  

In [2]:
from google.colab import drive

drive.mount('/content/drive')
path  = '/content/drive/MyDrive/Hackathon/'

Mounted at /content/drive


In [3]:
import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re

#Para encontrar la página en la que se encuentra el dato.
def fnPDF_FindText(xFile, xString):
    # xfile : the PDF file in which to look
    # xString : the string to look for
    # import pyPdf, re
    PageFound = -1
    pdfDoc = pyPdf.PdfFileReader(xFile, "rb")
    for i in range(1, pdfDoc.getNumPages()):
        content = ""
        content += pdfDoc.getPage(i-1).extractText() + "\n"
        content1 = content#.lower()#content.encode('ascii', 'ignore').lower()
        ResSearch = re.search(xString, content1)
        if ResSearch is not None:
           PageFound = i 
           break
    return PageFound

In [4]:
import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re

#Para crear una clave de trimestre válida
def create_trimester_key(file_name):
    seed = "20"   
    element = file_name.split("_")[2]
    trimester = element[1]+element[2]
    year = element[2:4]
    return seed + year + trimester

In [8]:
import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re

#Para hacer la extraccion (SOLO PARA ESTADO DE RESULTADOS)
def estado_de_resultados_extraccion(path,filename,short_description,keyword):
    pagina = fnPDF_FindText(path+filename,"Estado de resultados")
    print("      Archvo encontrado en la página: ", pagina)

    print("      Paso 1: Limpieza general.")  
    tabla_leer = camelot.read_pdf(path+archivo,flavor="stream",pages=str(pagina),edge_tol=20,row_tol=10)

    #Se eliminan las partes HORIZONATALES malas, para ello se hace la transpuesta y se trata sólo con "columnas".
    estado_de_resultados = tabla_leer[0].df.transpose()
    
    #El renglón 1 pasa a ser las "nuevas columnas", también se elimina un renglón con datos vacíos. 
    new_header = estado_de_resultados.iloc[0] #grab the first row for the header
    estado_de_resultados_2 = estado_de_resultados[1:] #take the data less the header row
    estado_de_resultados_2.columns = new_header #set the header row as the df header
    
    print("      Paso 2: Limpieza más profunda.")
    #Se eliminan todas las columnas con valores NULL y "" (cadena vacía)
    estado_de_resultados_2.replace('', np.nan, inplace = True)
    estado_de_resultados_2
    estado_de_resultados_2.dropna(how='all', axis=1,inplace = True)
    estado_de_resultados_2.replace('—', '0', inplace = True)
    estado_de_resultados_2.drop([""], axis=1, inplace=True)
  
    #Se regresa el conjunto a la forma transpuesta de nuevo porque ya habrá estado limpia y se continúa con la limpieza
    estado_de_resultados_2 = estado_de_resultados_2.T
    estado_de_resultados_2.drop(estado_de_resultados_2.columns[[3,4,7]],axis=1,inplace=True)
    estado_de_resultados_2.drop(estado_de_resultados_2.columns[5:],axis=1,inplace=True)

    print("      Paso 3: Imputar valores.")

    #Se agregan las columnas acordadas por el equipo.
    estado_de_resultados_2.columns = ['T_0_feature_value', 'T_1_feature_value',	'T_2_feature_value','T_3_feature_value','T_4_feature_value']

    #Se agrega el nombre del banco, de la tabla y el identificador del trimestre.
    estado_de_resultados_2["bank_name"] = "Santander"
    estado_de_resultados_2["table_name"] = "Estado de Resultados"

    #Caso especial para el archivo mencionado (se elimina una fila más)
    if filename == "Reporte_Resultados_2T20.pdf":
        estado_de_resultados_2 = estado_de_resultados_2.iloc[:-1 , :]

    estado_de_resultados_2["trimester_date"] = create_trimester_key(filename)
    estado_de_resultados_2.reset_index(inplace = True)

    #Se elimina una fila que había quedado vacía y se agrega el campo de la descripción con base en el catálogo.
    estado_de_resultados_2 = estado_de_resultados_2.iloc[1: , :]
    estado_de_resultados_2['short_description'] = short_description

    #Se elimina una columna vacía.    
    estado_de_resultados_2.drop(estado_de_resultados_2.columns[0],axis=1,inplace=True)

    #Por cada una de las columnas se imputan los caracteres "(" ")" "," el último porque marca error al convertirlo a flotante.
    columnas = estado_de_resultados_2.columns
    for columna in columnas:
        estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace("(","-") 
        estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace(")","")
        estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace(",","")
    
    #Nota: hasta este punto TODAS LAS VARIABLES quedan como string para que se puedan despues convertir a entero.
    return estado_de_resultados_2[["trimester_date","bank_name","table_name","short_description",
                                   "T_0_feature_value","T_1_feature_value","T_2_feature_value","T_3_feature_value","T_4_feature_value"]]

In [9]:
#Comienza el proceso

import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re


#De todas las columnas que mostró Samuel, éstas son las que Santander tiene EN EL ESTADO DE RESULTADOS.
"""
Ingreso por intereses
Ingresos por primas (neto)
Gastos por intereses
Incremento neto de reservas técnicas
Siniestralidad, reclamaciones y otras obligaciones contractuales (neto)
1 Margen Financiero 
2 Estimación preventiva para riesgos crediticios
3 Margen financiero ajustado por riesgos crediticios 
Ingreso no financiero
Comisiones y tarifas cobradas
4 Comisiones y tarifas pagadas (???????)
5 Resultado por intermediación 
6 Otros ingresos (egresos) de la operación
7 Gastos de administración y promoción
8 Rsultado de la operación
9 Participación en el resultado de subsidiarias no consolidadas y asociad
10 Resultado antes de impuestos a la utilidad
Impuestos a la utilidad causados
11 Impuestos a la utilidad diferidos (netos)
Resultado antes de participación no controladora
Operaciones discontinuadas
Participación no controladora
Resultado Neto
"""

#Se agregan éstas porque no estaban en el listado general. 
"""
12 Utilidad neta mayoritaria
13 Tasa efectiva de impuestos
"""


#Lista de las descripciones finales.
short_description = [
                     'Margen Financiero', 
                     'Estimación preventiva para riesgos crediticios',
                     'Margen financiero ajustado por riesgos crediticios',
                     'Comisiones y tarifas pagadas',
                     'Resultado por intermediación',
                     'Otros ingresos (egresos) de la operación',
                     'Gastos de administración y promoción',
                     'Resultado de la operación',
                     'Participación en el resultado de subsidiarias no consolidadas y asociad',
                     'Resultado antes de impuestos a la utilidad',
                     'Impuestos a la utilidad diferidos (netos)',
                     'Utilidad neta mayoritaria',
                     'Tasa efectiva de impuestos'
                    ] 

#Nombres de los archivos.
lista_archivos = [
                  "Reporte_Resultados_1T21.pdf",
                  #"Reporte_Resultados_2T20.pdf",
                  #"Reporte_Resultados_2T21.pdf",
                  #"Reporte_Resultados_3T20.pdf",
                  #"Reporte_Resultados_4T20.pdf"                    
                 ]



In [10]:
path  = "/content/drive/MyDrive/hackatonBBVA2021/estadosFinancieros/Santander/"
outputpath = "/content/drive/MyDrive/hackatonBBVA2021/csv/Santander/"
list_of_dataframes = []

for archivo in lista_archivos:
    print("******* Ahora ejecutando archivo: ", archivo)
    df_provisional = estado_de_resultados_extraccion(path,archivo,short_description,"Estado de resultados")
    list_of_dataframes.append(df_provisional)

final_dataframe = pd.concat(list_of_dataframes)
print(final_dataframe)
final_dataframe.to_csv(outputpath+"estado_de_resultados_santander.csv",index=False)

******* Ahora ejecutando archivo:  Reporte_Resultados_1T21.pdf
      Archvo encontrado en la página:  7
      Paso 1: Limpieza general.
      Paso 2: Limpieza más profunda.
      Paso 3: Imputar valores.
   trimester_date  bank_name  ... T_3_feature_value T_4_feature_value
1          2021T2  Santander  ...             15585             16896
2          2021T2  Santander  ...             -7075             -5165
3          2021T2  Santander  ...              8510             11731
4          2021T2  Santander  ...              4902              4697
5          2021T2  Santander  ...              1398               883
6          2021T2  Santander  ...              -664              -212
7          2021T2  Santander  ...             -9894             -9785
8          2021T2  Santander  ...              4252              7314
9          2021T2  Santander  ...                77                 4
10         2021T2  Santander  ...              4329              7318
11         2021T2  Santand

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:4389]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:28]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:4389]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:4174]


In [11]:
final_dataframe

,trimester_date,bank_name,table_name,short_description,T_0_feature_value,T_1_feature_value,T_2_feature_value,T_3_feature_value,T_4_feature_value
1,2021T2,Santander,Estado de Resultados,Margen Financiero,15585,16272,16896,15585,16896
2,2021T2,Santander,Estado de Resultados,Estimación preventiva para riesgos crediticios,-7075,-3152,-5165,-7075,-5165
3,2021T2,Santander,Estado de Resultados,Margen financiero ajustado por riesgos crediti...,8510,13120,11731,8510,11731
4,2021T2,Santander,Estado de Resultados,Comisiones y tarifas pagadas,4902,4709,4697,4902,4697
5,2021T2,Santander,Estado de Resultados,Resultado por intermediación,1398,723,883,1398,883
6,2021T2,Santander,Estado de Resultados,Otros ingresos -egresos de la operación,-664,-379,-212,-664,-212
7,2021T2,Santander,Estado de Resultados,Gastos de administración y promoción,-9894,-11102,-9785,-9894,-9785
8,2021T2,Santander,Estado de Resultados,Resultado de la operación,4252,7071,7314,4252,7314
9,2021T2,Santander,Estado de Resultados,Participación en el resultado de subsidiarias ...,77,79,4,77,4
10,2021T2,Santander,Estado de Resultados,Resultado antes de impuestos a la utilidad,4329,7150,7318,4329,7318
